In [ ]:
import pandas as pd
import numpy as np

# set column fixed widths
col_widths = [14] + [9]*12

# read the .OUT file, skipping the first 13 rows of station info
df = pd.read_fwf(
    '/s3/scratch/jamie.towner/flood_aa/zimbabwe/data/observations/raw_data/F22D.OUT',
    widths=col_widths,
)

# get month column names
columns = ['Day'] + list(df.iloc[8].values[1:]) + ['Year']

# add year column
df['Year'] = ""
for l in range(5,len(df),51):
    start = l+4
    end = l+4+31
    df.loc[start:end,'Year'] = "".join(df.iloc[l].values[6:8])

# read the first 31 out of every 52 rows
def get_rows_to_skip(df, n, start, end, flen):
    rows_to_skip = [i for i in range(flen) if i % n < start or i % n >= end]
    return rows_to_skip

n = 51
start = 9
end = 40
flen = len(df)

skiprows = get_rows_to_skip(df, n, start, end, flen)

# get only rows that are not skipped
df2 = df[~df.index.isin(skiprows)]

# set column name to months and index to year and day
df2.columns = columns
df2 = df2.set_index(['Year','Day'])

df2.head(40)

In [ ]:
# Replace '*' with NaN (assuming df2 is your dataframe)
df2.replace('*', np.nan, inplace=True)

# Step 1: Reset the index to make 'Day' a column
df2_reset = df2.reset_index()

# Step 2: Melt the DataFrame to long format
df_long = df2_reset.melt(id_vars=['Year', 'Day'], var_name='Month', value_name='Discharge')

# Step 3: Map month names to numbers, starting from October (OCT = 10, etc.)
month_map = {
    'OCT': 10, 'NOV': 11, 'DEC': 12, 'JAN': 1, 'FEB': 2, 'MAR': 3,
    'APR': 4, 'MAY': 5, 'JUN': 6, 'JUL': 7, 'AUG': 8, 'SEP': 9
}
df_long['Month_num'] = df_long['Month'].map(month_map)

# Step 4: Extract the starting year from the "Year" column
df_long['Start_Year'] = df_long['Year'].str.split('/').str[0].astype(int)

# Step 5: Adjust the year for Jan-Sep (shift by 1 year)
df_long['Adjusted_Year'] = df_long.apply(lambda row: row['Start_Year'] if row['Month_num'] >= 10 else row['Start_Year'] + 1, axis=1)

# Step 6: Manually set the first date as 01/10/1959 and handle subsequent dates.
def create_date(row):
    if row['Month_num'] == 10 and row['Day'] == 1 and row['Adjusted_Year'] == 1959:
        # Explicitly set the first date
        return pd.to_datetime("1959-10-01")
    else:
        try:
            # Calculate the exact date based on the Year, Month, and Day
            date_str = f"{row['Adjusted_Year']}-{row['Month_num']}-{row['Day']}"
            date = pd.to_datetime(date_str, errors='coerce')  # Handle invalid dates
            return date
        except Exception as e:
            print(f"Error with row {row}: {e}")
            return pd.NaT  # Return NaT for invalid rows

# Step 7: Apply the function to create the date column
df_long['Date'] = df_long.apply(create_date, axis=1)

# Step 8: Remove rows where Date is NaT (invalid date rows)
df_long = df_long.dropna(subset=['Date'])

# Step 9: Sort by Date to ensure chronological order
df_long.sort_values(by='Date', inplace=True)

# Step 10: Reset index after sorting
df_long.reset_index(drop=True, inplace=True)

pd.set_option('display.max_rows', 5000)
# Step 11: Print the first few rows to verify
#print(df_long[['Date', 'Discharge']].head(100))

df_long = df_long[['Date', 'Discharge']]

In [ ]:
# Get the first date in df_long
#start_date = df_long['Date'].min()
start_date = '2003-01-01'
end_date = '2023-12-31'

# Define full date range from start to end
full_date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Create a new DataFrame with the full date range
df_complete = pd.DataFrame({'Date': full_date_range})

# Select only the necessary columns from the original DataFrame
df_filtered = df_long[['Date', 'Discharge']]

# Merge to insert missing dates and fill with NaN
df_filtered = pd.merge(df_complete, df_filtered, on='Date', how='left')


In [ ]:
# Define full date range from start to end
full_date_range = pd.date_range(start='2003-01-01', end='2023-12-31', freq='D')

# Filter to include only October–April months
oct_apr_range = full_date_range[full_date_range.month.isin([10, 11, 12, 1, 2, 3, 4])]

# Create a new DataFrame with the full Oct–Apr date range
df_complete = pd.DataFrame({'Date': oct_apr_range})

# Merge with your filtered df_long to fill missing dates with NaN
df_filtered = df_long[['Date', 'Discharge']]
df_filtered = pd.merge(df_complete, df_filtered, on='Date', how='left')

In [ ]:
df_filtered.loc[:, 'Discharge'] = pd.to_numeric(df_filtered['Discharge'], errors='coerce')
count = (df_filtered['Discharge'] >= 461.6 ).sum()
print(f"Number of times 'Discharge' >= thresold: {count}")

In [ ]:
# Total number of values in the 'Discharge' column
total_values = df_filtered['Discharge'].size

# Total number of missing values in the 'Discharge' column
missing_values = df_filtered['Discharge'].isna().sum()

# Percentage of missing values
missing_percentage = (missing_values / total_values) * 100

result = 100 - missing_percentage

print(f"Percentage of : {result:.2f}%")

In [ ]:
df_filtered.tail(50)

In [ ]:
output_path = "/s3/scratch/jamie.towner/flood_aa/zimbabwe/data/observations/gauging_stations/katiyo.csv"
df_long.to_csv(output_path, index=False)